In [2]:
%load_ext Cython

In [57]:
%%cython
from libc.math cimport abs, sqrt, ceil, floor, log, log2, acos, cos
from numpy cimport ndarray
import numpy as np
from numpy import ones, zeros, int32, float32, uint8, fromstring
from numpy import sort, empty, array, arange, concatenate, searchsorted
from numpy import minimum, maximum,divide, std, mean
from numpy import min as nmin
from numpy import max as nmax
import h5py as h

def pairRegionsIntersection(ndarray[int, ndim=2] pairs,
                            ndarray[int, ndim=2] regions,
                            exclude=False, allow_partial=False,
                            bool_out=False):
    '''
    Given an list of pairs of the form [a,b] where a<b and regions of the form [c,d] we want to return the indices
    (within the pair list) of pairs which overlap with the regions. This will be used to exclude certain regions of the
    genome if needed.
     
    Arguments:
    
    - pairs: (N,2) shape array where each row details a pair.
    - regions: (M,2) shape array where each row details a region.
    - exclude: Boolean. If true, then return the indices of pairs which don't overlap with any of the regions.
    - allow_partial: Boolean. If true then include pair indices which only partially overlap with a given region.
                     Here we consider the 'partial' to apply to the pair. That is, if allow_partial == False then
                     we must satisfy either:
                         - the entire pair is included in a region if exclude == False
                         - The entire pair must not be included in a region if exclude == True
                     If allow_partial == True then we must satisfy:
                         - At least part of the pair touches at least one region if exclude == False
                         - At least part of the pair touches no regions if exclude == True
                      
    Returns:
    
    - indices: The indices of the pairs in the pair array which satisfy the overlapping conditions.
    '''
    cdef int i, j, k, a, b
    cdef int exc = int(exclude)
    cdef int partial = int(allow_partial)
    cdef int bool_o = int(bool_out)
    cdef int ni = 0
    cdef int np = len(pairs)
    cdef int nr = len(regions)
    cdef ndarray[int, ndim=1] indices = zeros(np, int32)
    cdef ndarray[int, ndim=1] indices_out = empty(np, int32)
    cdef ndarray[int, ndim=1] order = array(regions[:,0].argsort(), int32)
    cdef ndarray[int, ndim=2] regs = empty((nr+1,2), int32)
    cdef minpoint = minimum(nmin(pairs), nmin(regions))
    cdef maxpoint = maximum(nmax(pairs), nmax(regions))

    if exc:
        for i in range(nr):
            regs[i,1] = regions[order[i],0]
        regs[0,0] = minpoint - 1
        regs[-1,1] = maxpoint + 1
    else:
        regs = regions
    
    
    for i in range(np):
        for k in range(nr):
            j = order[k]
            if (regs[j,1] > pairs[i,0]) and (pairs[i,1] > regs[j,0]):
                #Pair is at least partially overlapping with the region
                if partial:
                    indices[i] = 1
                    if bool_o:
                        return 1
                    continue
                elif (regs[j,1] >= pairs[i,1]) and (pairs[i,0] >= regs[j,0]):
                    #Pair is entirely containing within the region
                    indices[i] = 1
                    if bool_o:
                        return 1
                    continue
    
    if bool_o:
        return 0
    
    for i in range(np):
        if indices[i] == 1:
            indices_out[ni] = i
            ni +=1
    
    return indices_out[:ni]

def multi_pairRegionsIntersection(ndarray[int, ndim=3] pairs,
                                  ndarray[int, ndim=3] regions,
                                  exclude=False, allow_partial=False, indices = False):
    '''
    Given an list of pairs of the form [a,b] where a<b and regions of the form [c,d] we want to work pairwise to
    return the region indices of the regions which overlap with each pair.
     
    Arguments:
    
    - pairs: (N,2) shape array where each row details a pair.
    - regions: (M,2) shape array where each row details a region.
    - exclude: Boolean. If true, then return the indices of pairs which don't overlap with any of the regions.
    - allow_partial: Boolean. If true then include pair indices which only partially overlap with a given region.
                     Here we consider the 'partial' to apply to the pair. That is, if allow_partial == False then
                     we must satisfy either:
                         - the entire pair is included in a region if exclude == False
                         - The entire pair must not be included in a region if exclude == True
                     If allow_partial == True then we must satisfy:
                         - At least part of the pair touches at least one region if exclude == False
                         - At least part of the pair touches no regions if exclude == True
                      
    Returns:
    
    - out: A maximum of an (N*M,2) shape array where each column details a pair group index and a regions group
           index where than pair group and that region group overlap. This is essentially gonna be in COO
           format
    '''
    cdef int i, j, k, np, nr,nl
    cdef int overlap
    cdef int ind = int(indices)
    cdef int maxnp = len(pairs[0,:,0]) 
    cdef int maxnr = len(regions[0,:,0])
    cdef int minpairs = nmin(pairs)
    cdef int minregions = nmin(regions)
    cdef int npairgroups = len(pairs)
    cdef int nreggroups = len(regions)
    cdef ndarray[int, ndim=2] out = zeros((npairgroups*nreggroups,2), int32)
    
    for i in range(npairgroups):
        print("#################################")
        print("pair {}".format(i))
        print("Unedited pairs:")
        print(pairs[i,:,:])
        np = maxnp
        for k in range(maxnp):
            if pairs[i,k,0] == minpairs:
                np = k
                break
        for j in range(nreggroups):
            print("#################################")
            print("region {}".format(j))
            print("Unedited regions:")
            print(regions[j,:,:])
            nr = maxnr
            for k in range(maxnr):
                if regions[j,k,0] == minregions:
                    nr = k
                    break
            print("******************************")
            print("Edited pairs:")
            print(pairs[i,:np,:])
            print("Edited regions:")
            print(regions[j,:nr,:])
            overlap = pairRegionsIntersection(pairs[i,:np,:],
                                              regions[j,:nr,:],
                                              allow_partial = allow_partial,
                                              exclude = exclude,
                                              bool_out = True)
            print("Output: {}".format(overlap))
            if overlap == 1:
                out[nl,0] = i
                out[nl,1] = j
                nl +=1
    
    return out[:nl,:]
                



In [1]:
import numpy as np
pairs = np.array([[[10,50],
                   [51,52],
                   [55,65],
                   [70,100]],[[5,20],
                              [21,22],
                              [-1,-1],
                              [-1,-1]],[[110,130],
                                        [-1,-1],
                                        [-1,-1],
                                        [-1,-1]]]).astype('int32')
regions = np.array([[[0,5],
                     [7,9],
                     [-1,-1],
                     [-1,-1],
                     [-1,-1]],[[2,7],
                               [10,22],
                               [7,17],
                               [21,100],
                               [105,151]],[[10,20],
                                           [21,22],
                                           [34,47],
                                           [51,100],
                                           [101,130]],[[10,20],
                                                       [30,40],
                                                       [50,60],
                                                       [-1,-1],
                                                       [-1,-1]]]).astype('int32')

In [2]:
pairs.shape

(3, 4, 2)

In [62]:
multi_pairRegionsIntersection(pairs, regions)

#################################
pair 0
Unedited pairs:
[[ 10  50]
 [ 51  52]
 [ 55  65]
 [ 70 100]]
#################################
region 0
Unedited regions:
[[ 0  5]
 [ 7  9]
 [-1 -1]
 [-1 -1]
 [-1 -1]]
******************************
Edited pairs:
[[ 10  50]
 [ 51  52]
 [ 55  65]
 [ 70 100]]
Edited regions:
[[0 5]
 [7 9]]
Output: 0
#################################
region 1
Unedited regions:
[[  2   7]
 [ 10  22]
 [  7  17]
 [ 21 100]
 [105 151]]
******************************
Edited pairs:
[[ 10  50]
 [ 51  52]
 [ 55  65]
 [ 70 100]]
Edited regions:
[[  2   7]
 [ 10  22]
 [  7  17]
 [ 21 100]
 [105 151]]
Output: 1
#################################
region 2
Unedited regions:
[[ 10  20]
 [ 21  22]
 [ 34  47]
 [ 51 100]
 [101 130]]
******************************
Edited pairs:
[[ 10  50]
 [ 51  52]
 [ 55  65]
 [ 70 100]]
Edited regions:
[[ 10  20]
 [ 21  22]
 [ 34  47]
 [ 51 100]
 [101 130]]
Output: 1
#################################
region 3
Unedited regions:
[[10 20]
 [30 40]
 [5

array([[0, 1],
       [0, 2],
       [0, 3],
       [1, 1],
       [1, 2],
       [2, 1],
       [2, 2]], dtype=int32)

In [129]:
%%cython
from libc.math cimport abs, sqrt, ceil, floor, log, log2, acos, cos
from numpy cimport ndarray
import numpy as np
from numpy import ones, zeros, int32, float32, uint8, fromstring
from numpy import sort, empty, array, arange, concatenate, searchsorted
from numpy import minimum, maximum,divide, std, mean
from numpy import min as nmin
from numpy import max as nmax
import h5py as h

def non_overlapping(ndarray[int, ndim=2] regions):
    '''
    Given a list of regions which may be overlapping we return a sorted list of regions which aren't overlapping
    arguments:
    - regions: An (N,2) shape array with each row detailing the start and end of a region
    
    outputs:
    - nonoverlapping: An (M,2) shape array where each row details a covered region. The regions in the output array
                      will be sorted by start position and non-overlapping with each other. 
    '''
    cdef int idx, jdx, nr = len(regions)
    cdef ndarray[int, ndim=2] outregs = zeros((nr,2),int32)
    cdef ndarray[int, ndim=2] values = zeros((2*regions.shape[0],2), int32)
    for idx in range(nr):
        values[idx,0] = regions[idx,0]
        values[idx,1] = 1
        values[idx+nr,0] = regions[idx,1]
        values[idx+nr,1] = -1
    cdef ndarray[int, ndim=1] order = array(values[:,0].argsort(), int32)
    
    cdef int total = 0
    cdef int new_point = 0
    cdef int valtype = 0
    cdef int current_start = 0
    cdef int current_end = 0
    cdef int out_idx = 0
    for idx in np.arange(2*nr):
        jdx = order[idx]
        new_point, valtype = values[jdx]
        total += valtype
        if valtype <0:
            current_end = new_point

        if (total == 1) and valtype > 0:
            current_start = new_point
                
        if total == 0:
            outregs[out_idx,0] = current_start
            outregs[out_idx,1] = current_end
            out_idx += 1
            
    return outregs[:out_idx,:]
                


def pairRegionsOverlap(ndarray[int, ndim=2] pairs,
                       ndarray[int, ndim=2] regions,
                       exclude=False):

    '''
    Given some (N,2) shape array of pairs a,b with a<b and an (M,2) shape array of regions c,d, with c<d, return a new
    (R,2) shape array of overlaps e,f with e<f such that each overlap is the intersection of the sets P and R where P
    is the union of the pairs and R is the union of the regions. 
    
    Arguments:
    
    - pairs: (N,2) shape array where each row details a pair.
    - regions: (M,2) shape array where each row details a region.
    - exclude: Boolean. If True then the function returns a new set of overlaps composed of the overlaps in the set
               P union ~R. 
               
    Returns:
    - overlap_bounds: An (R,2) array detailing the set of intervals making up the overlaps between the pairs and
                      regions
    '''
    
    pairs = non_overlapping(pairs)
    regions = non_overlapping(regions)
    cdef int i, j, k
    cdef int exc = int(exclude)
    #totaloverlaps index counter
    cdef int tnio = 0
    #number of pairs
    cdef int np = len(pairs)
    #number of regions
    cdef int nr = len(regions)
    #order array for the regions
    cdef ndarray[int, ndim=1] order = array(regions[:,0].argsort(), int32)
    #Output regions array (maximum number of possible regions would be np + nr -1
    cdef ndarray[int, ndim=2] overlap_bounds = empty(((np + nr),2), int32)
    cdef ndarray[int, ndim=1] current_pair = empty(2, int32)
    
    for i in range(np):
        current_pair = pairs[i,:]
        
        if current_pair[1] < regions[order[0],0]:
            if exc:
                overlap_bounds[tnio,:] = current_pair
                tnio += 1
                
            continue
        
        if current_pair[0] > regions[order[nr-1],1]:
            if exc:
                overlap_bounds[tnio,:] = current_pair
                tnio += 1
                
            continue
            
        for k in range(nr):
            j = order[k]
            #print(i,current_pair, j, regions[j], tnio)
            if regions[j,0] < current_pair[1] and regions[j,1] > current_pair[0]:
                #There is an overlap between this region and this pair chunk
                if regions[j,0] <= current_pair[0] and current_pair[0] < regions[j,1]:
                    #Start of the current pair chunk is in the region
                    if regions[j,1] >= current_pair[1]:
                        #Pair chunk entirely contained with a region 
                        if exc:
                            #If exclude then we know that this
                            #pair can't be included - it is contained within a 
                            #region so break the current loop and move onto the 
                            #next pair
                            break
                        else:
                            #If include then we want to add this pair chunk to the 
                            #list of output pair chunks
                            overlap_bounds[tnio] = current_pair
                            tnio += 1
                            continue
                    else:
                        #Start of pair chunk is in the region, end isnt
                        if exc:
                            #If exclude then we just shave off the bit of this
                            #pair that touches the reegion and continue to the 
                            #next region
                            current_pair[0] = regions[j,1]
                            continue
                        else:
                            #If include then we add this overlap to the 
                            #output and shave off the bit of the pair that overlaps
                            #with the region and then continue on to the next region
                            overlap_bounds[tnio,0] = current_pair[0]
                            overlap_bounds[tnio,1] = regions[j,1]
                            
                            tnio += 1
                            
                            current_pair[0] = regions[j,1]
                            continue
                if regions[j,0] > current_pair[0]:
                    #Start of the current pair chunk is lower than the start of
                    #the region
                    if regions[j,1] < current_pair[1]:
                        #End of the current pair chunk is higher than the start of the
                        #regions - i.e. region entirely contained within current
                        #pair chunk
                        if exc:
                            #Since the regions are sorted we know that the lower portion
                            #of the pair which is cutoff can't overlap with any other 
                            #region so we add it to the output
                            overlap_bounds[tnio,0] = current_pair[0]
                            overlap_bounds[tnio,1] = regions[j,0]
                            tnio += 1
                            
                            #We then shave the current pair to only include the bit
                            #which didn't overlap with the current region above
                            current_pair[0] = regions[j,1]
                            continue
                        else:
                            #We include the overlapping region
                            overlap_bounds[tnio,:] = regions[j,:]
                            tnio += 1
                            
                            #Then shave the current pair
                            current_pair[0] = regions[j,1]
                            continue
                    else:
                        #Current pair end must overlap with the region
                        if exc:
                            overlap_bounds[tnio,0] = current_pair[0]
                            overlap_bounds[tnio,1] = regions[j,0]
                            tnio +=1
                            #Since the regions are sorted we know that
                            #we've already got all the info out
                            #of this pair
                            current_pair[0] = regions[j,0]
                            break
                        else:
                            overlap_bounds[tnio,0] = regions[j,0]
                            overlap_bounds[tnio,1] = current_pair[1]
                            tnio += 1
                            #Since the regions are sorted we know that
                            #there won't be any more of this pair that
                            #is overlapping any regions so continue
                            #to the next pair
                            current_pair[0] = regions[j,0]
                            break
            elif exc and regions[j,0] > current_pair[1]:
                overlap_bounds[tnio,:] = current_pair 
                tnio += 1
                break
                    
    
    return overlap_bounds[:tnio]

In [130]:
pairs = np.array([[10,20],
                   [30,40],
                   [50,60],
                   [70,90]]).astype('int32')
regions = np.array([[15,25],
                    [26,35],
                    [38,55],
                    [55,57],
                    [75,151]]).astype('int32')

In [131]:
pairRegionsOverlap(pairs,regions, exclude = True)

array([[10, 15],
       [35, 38],
       [57, 60],
       [70, 75]], dtype=int32)

In [132]:
data_root = "/disk1/dh486/Data"
chip_paths = {'H3K4me3':'{}/ChIPseq/0hrs/peaks_bed/old_haploid/GSEXXXXXLaue_H3K4me3_hapmESC_peaks.npz'.format(data_root),
              'H3K27me3':'{}/ChIPseq/0hrs/peaks_bed/old_haploid/GSEXXXXXLaue_H3K27me3_hapmESC_peaks.npz'.format(data_root),
              'H3K27ac':'{}/ChIPseq/0hrs/peaks_bed/GSE56098_H3K27ac_ESC_peaks.npz'.format(data_root),
              'H3K36me3':'{}/ChIPseq/0hrs/peaks_bed/old_haploid/GSEXXXXXLaue_H3K36me3_hapmESC_peaks.npz'.format(data_root)}

In [133]:
x1 = dict(np.load(chip_paths['H3K4me3']))['dataTrack/regions/H3K4me3/1']
x2 = dict(np.load(chip_paths['H3K27me3']))['dataTrack/regions/H3K27me3/1']

In [134]:
pairRegionsOverlap(x1.astype('int32'),x2.astype('int32'), exclude = True)

array([[  3670867,   3671078],
       [  3671153,   3671499],
       [  3671734,   3671914],
       ...,
       [195176428, 195176645],
       [195213000, 195213215],
       [195241698, 195241961]], dtype=int32)